### Import Lib

In [1]:
%pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [15]:
import numpy as np
import pandas as pd
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Layer
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
from sklearn.model_selection import train_test_split
import gradio as gr

tf.get_logger().setLevel('ERROR')

In [3]:
### Configurations

In [4]:
seed = 42
class_names = [0,1]
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]
map_name_to_handle = {'small_bert/bert_en_uncased_L-4_H-512_A-8': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'}
map_model_to_preprocess = {'small_bert/bert_en_uncased_L-4_H-512_A-8': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'}

In [5]:
tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [6]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [7]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [8]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = [tf.metrics.BinaryAccuracy(), f1_m, precision_m, recall_m]
callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_f1_m', patience = 3)

In [16]:
class attention(Layer):
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences

        super(attention,self).__init__()

    def build(self, input_shape):
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1), initializer="zeros")
        super(attention,self).build(input_shape)


    def call(self, x):
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        if self.return_sequences:
            return output
        return K.sum(output, axis=1)

In [11]:
def build_LSTM():
    n_unique_words = 29603  # cut texts after this number of words
    maxlen = 250
    batch_size = 128
    model = Sequential()
    model.add(Embedding(n_unique_words, 128, input_length=maxlen))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=metrics) 
    return model

In [12]:
def pipeline(text):
    x = bert_preprocess_model([text])['input_word_ids']

    maxlen = 250
    batch_size = 128
    x = sequence.pad_sequences(x, maxlen=maxlen)
    model = build_LSTM()
    model.load_weights("weights/polarity.ckpt")
    y = model.predict(x)

    return int(y[0]>=0.5)

In [13]:
pipeline('gun control ')

1

In [17]:
iface = gr.Interface(
  fn = pipeline,
  inputs = gr.Textbox(lines=2, placeholder="What is your opinion about gun laws in USA?"),
  outputs = "text"
  )
iface.launch(share = True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://94e8bb8cd4315304.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x20211c887b8>,
 'http://127.0.0.1:7861/',
 'https://94e8bb8cd4315304.gradio.app')

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 405, in run_asgi
    self.scope, self.receive, self.send
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\fastapi\applications.py", line 270, in __call__
    await super().__call__(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\starlette\applications.py", line 124, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\starlette\middleware\errors.py", line 184, in __call__
    raise exc
  File "c:\users\scse-cil\appdata\

ERROR:uvicorn.error:Exception in ASGI application
Traceback (most recent call last):
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 405, in run_asgi
    self.scope, self.receive, self.send
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\fastapi\applications.py", line 270, in __call__
    await super().__call__(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\starlette\applications.py", line 124, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\starlette\middleware\errors.py", line 184, in __call__
    raise exc
  File "c:\users\scse-ci

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 405, in run_asgi
    self.scope, self.receive, self.send
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\fastapi\applications.py", line 270, in __call__
    await super().__call__(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\starlette\applications.py", line 124, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\starlette\middleware\errors.py", line 184, in __call__
    raise exc
  File "c:\users\scse-cil\appdata\

...

ERROR:uvicorn.error:Exception in ASGI application
Traceback (most recent call last):
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 405, in run_asgi
    self.scope, self.receive, self.send
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\fastapi\applications.py", line 270, in __call__
    await super().__call__(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\starlette\applications.py", line 124, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\starlette\middleware\errors.py", line 184, in __call__
    raise exc
  File "c:\users\scse-ci

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 405, in run_asgi
    self.scope, self.receive, self.send
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\fastapi\applications.py", line 270, in __call__
    await super().__call__(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\starlette\applications.py", line 124, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\starlette\middleware\errors.py", line 184, in __call__
    raise exc
  File "c:\users\scse-cil\appdata\

ERROR:uvicorn.error:Exception in ASGI application
Traceback (most recent call last):
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 405, in run_asgi
    self.scope, self.receive, self.send
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\fastapi\applications.py", line 270, in __call__
    await super().__call__(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\starlette\applications.py", line 124, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\starlette\middleware\errors.py", line 184, in __call__
    raise exc
  File "c:\users\scse-ci

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 405, in run_asgi
    self.scope, self.receive, self.send
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\fastapi\applications.py", line 270, in __call__
    await super().__call__(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\starlette\applications.py", line 124, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\starlette\middleware\errors.py", line 184, in __call__
    raise exc
  File "c:\users\scse-cil\appdata\

ERROR:uvicorn.error:Exception in ASGI application
Traceback (most recent call last):
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 405, in run_asgi
    self.scope, self.receive, self.send
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\fastapi\applications.py", line 270, in __call__
    await super().__call__(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\starlette\applications.py", line 124, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\users\scse-cil\appdata\local\programs\python\python37\lib\site-packages\starlette\middleware\errors.py", line 184, in __call__
    raise exc
  File "c:\users\scse-ci